In [13]:
import glob
import os
import cv2
import imutils
import argparse
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, Conv3DTranspose
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [14]:
# Function to store image in array
def store_inarray(image_path, lst):
    image = load_img(image_path)
    image = img_to_array(image)
    image = cv2.resize(image, (227, 227), interpolation=cv2.INTER_AREA)
    gray = 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]
    lst.append(gray)

In [18]:
# Paths
train_path = r'D:/Omkar/Fourth Year/Major Project 2/Main/Avenue_Dataset/Avenue Dataset/training_videos'
fps = 5
train_images_path = os.path.join(train_path, 'frames')
os.makedirs(train_images_path, exist_ok=True)

In [19]:
train_videos = glob.glob(os.path.join(train_path, '*.avi'))
store_image_train = []

if not os.listdir(train_images_path):
    for video_path in train_videos:
        video_name = os.path.basename(video_path)
        video_folder = os.path.join(train_images_path, video_name[:-4])
        os.makedirs(video_folder, exist_ok=True)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        frame_count = 0

        # Read frames until the video is complete
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save frame as an image file
            frame_path = os.path.join(video_folder, '{:03d}.jpg'.format(frame_count))
            cv2.imwrite(frame_path, frame)

            store_inarray(frame_path, store_image_train)

            frame_count += 1

        cap.release()
else:
    print("flames are already avalable")
# Print the number of training frames extracted
print("Training frames extracted:", len(store_image_train))


Training frames extracted: 15328


In [20]:
if len(store_image_train) > 0:
    store_image_train = np.array(store_image_train)
    a, b, c = store_image_train.shape

    store_image_train.resize(b, c, a)
    store_image_train = (store_image_train - store_image_train.mean()) / store_image_train.std()
    store_image_train = np.clip(store_image_train, 0, 1)
    np.save('training.npy', store_image_train)
    print("Training images saved as numpy array.")
else:
    print("No training images found.")


Training images saved as numpy array.


In [21]:
# Load the training and testing datasets
training_data = np.load('training.npy')

In [22]:
print("Training data shape:", training_data.shape)

Training data shape: (227, 227, 15328)


In [23]:
frames_train = training_data.shape[2]
frames_train = frames_train - frames_train % 10
training_data = training_data[:, :, :frames_train]
training_data = training_data.reshape(-1, 227, 227, 10)
training_data = np.expand_dims(training_data, axis=4)

In [24]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

C:\Users\omkar\AppData\Roaming\Python\Python310\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [25]:
epochs=10
batch_size=1
callback_save = ModelCheckpoint("saved_model.keras", monitor="mean_squared_error", save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [26]:
target_data=training_data.copy()
stae_model.fit(training_data, target_data, batch_size=batch_size, epochs=epochs, callbacks=[callback_save, callback_early_stopping])

Epoch 1/10
1532/1532 ━━━━━━━━━━━━━━━━━━━━ 962s 623ms/step - accuracy: 0.6856 - loss: 0.1085
Epoch 2/10


C:\Users\omkar\AppData\Roaming\Python\Python310\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with mean_squared_error available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
C:\Users\omkar\AppData\Roaming\Python\Python310\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)


 311/1532 ━━━━━━━━━━━━━━━━━━━━ 12:10 598ms/step - accuracy: 0.7354 - loss: 0.0704

KeyboardInterrupt: 

In [33]:
stae_model.save("saved_model_final.keras")

In [34]:
stae_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d_2 (Conv3D)               │ (None, 55, 55, 10,     │        15,616 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_3 (Conv3D)               │ (None, 26, 26, 10, 64) │       204,864 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_3 (ConvLSTM2D)      │ (None, 26, 26, 10, 64) │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_4 (ConvLSTM2D)      │ (None, 26, 26, 10, 32) │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv_lstm2d_5 (ConvLSTM2D)      │ (None, 26, 26, 10, 64) │       221,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_2              │ (None, 55, 55, 10,     │       204,928 │
│ (Conv3DTranspose)               │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3d_transpose_3              │ (None, 227, 227, 10,   │        15,489 │
│ (Conv3DTranspose)               │ 1)                     │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,204,677 (12.22 MB)

 Trainable params: 1,068,225 (4.07 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2,136,452 (8.15 MB)

In [19]:
import cv2
import numpy as np
from keras.models import load_model
import imutils
import psutil
import time

In [21]:
path = 'D:/Omkar/Fourth Year/Major Project 2/Main/Avenue_Dataset/Avenue Dataset/testing_videos/12.avi'

detection()

Failed to read a frame
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.0003280696439818361
Current Memory Usage: 24.25 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
0.0003276864353967775
Current Memory Usage: 24.45 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
0.0003276863506610382
Current Memory Usage: 24.45 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 566ms/step
0.0003273166211342725
Current Memory Usage: 24.4 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 555ms/step
0.00032758941534690426
Current Memory Usage: 24.4 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step
0.00032763005863836227
Current Memory Usage: 24.4 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 459ms/step
0.0003275078373907838
Current Memory Usage: 24.4 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 450ms/step
0.0003273958985258345
Current Memory Usage: 24.45 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 539ms/step
0.0003273581708384166
Current Memory Usage: 24.45 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step
0.0003274843578493809
Current Memory Usage: 24.45 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step
0.0003276499691124768
Current Mem

In [13]:
def weapon_detect(path):
    # Read the model and configuration
    net = cv2.dnn.readNet("yolov3_training_2000.weights", "yolov3_testing.cfg")

    # Set preferable backend and target to GPU
    net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    classes = ["Weapon"]
    output_layer_names = net.getUnconnectedOutLayersNames()
    colors = np.random.uniform(0, 255, size=(len(classes), 3))

    cap = cv2.VideoCapture(path)
    skip_frames = 15  # Adjust the number of frames to skip

    frame_count = 0
    temp = 0  # Initialize temp to 0

    # Create the directory to save frames if it doesn't exist
    extracted_frames_dir = "D:/Omkar/Fourth Year/Major Project 2/Main/Intelligent_video_surveillance-main/Intelligent_video_surveillance-main/Extracted frames"
    if not os.path.exists(extracted_frames_dir):
        os.makedirs(extracted_frames_dir)

    # Define the constant filename
    frame_filename = os.path.join(extracted_frames_dir, "detected_frame.jpg")

    while True:
        ret, img = cap.read()
        if not ret:
            print("Failed to read a frame")
            break
        frame_count += 1
        if frame_count % skip_frames != 0:
            continue  # Skip frames

        height, width, channels = img.shape

        # Resize input image
        blob = cv2.dnn.blobFromImage(img, 0.00392, (320, 320), (0, 0, 0), True, crop=False)

        net.setInput(blob)
        outs = net.forward(output_layer_names)

        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)

                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)

                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

        if len(indexes) > 0:
            print("Weapon detected in frame")
            # Overwrite the frame with weapon detection
            cv2.imwrite(frame_filename, img)
            temp = 1  # Set temp to 1 if weapon is detected
            break  # Exit the loop if weapon is detected

        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
                cv2.putText(img, label, (x, y + 30), font, 3, color, 3)

        cv2.imshow("Image", img)
        key = cv2.waitKey(1)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    
    return temp  # Return the value of temp



In [20]:
def abnormal_action(path):
    def mean_squared_loss(x1, x2):
        difference = x1 - x2
        a, b, c, d, e = difference.shape
        n_samples = a * b * c * d * e
        sq_difference = difference**2
        Sum = sq_difference.sum()
        distance = np.sqrt(Sum)
        mean_distance = distance / n_samples
        return mean_distance

    # Load the model
    model = load_model("saved_model.keras")

    cap = cv2.VideoCapture(path)
    if not cap.isOpened():
        print("Error: Unable to open video capture.")
        return

    # Variables for resource monitoring
    cpu_percent_list = []
    memory_percent_list = []
    start_time = time.time()

    # Create the directory to save frames if it doesn't exist
    extracted_frames_dir = "D:/Omkar/Fourth Year/Major Project 2/Main/Intelligent_video_surveillance-main/Intelligent_video_surveillance-main/Extracted frames"
    if not os.path.exists(extracted_frames_dir):
        os.makedirs(extracted_frames_dir)

    # Define the constant filename
    frame_filename = os.path.join(extracted_frames_dir, "detected_frame.jpg")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or error while reading frames.")
            break

        imagedump = []
        for i in range(10):
            ret, frame = cap.read()
            if not ret:
                print("End of video or error while reading frames.")
                break

            image = imutils.resize(frame, width=700, height=600)
            frame = cv2.resize(frame, (227, 227), interpolation=cv2.INTER_AREA)
            gray = 0.2989 * frame[:, :, 0] + 0.5870 * frame[:, :, 1] + 0.1140 * frame[:, :, 2]
            gray = (gray - gray.mean()) / gray.std()
            gray = np.clip(gray, 0, 1)
            imagedump.append(gray)

        if not ret:
            break

        imagedump = np.array(imagedump)
        imagedump.resize(227, 227, 10)
        imagedump = np.expand_dims(imagedump, axis=0)
        imagedump = np.expand_dims(imagedump, axis=4)

        output = model.predict(imagedump)

        loss = mean_squared_loss(imagedump, output)
        print(loss)

        if loss > 0.000335:
            print('Abnormal Event Detected')
            cv2.putText(image, "Abnormal Event", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)
            cv2.imwrite(frame_filename, image)
            # Overwrite the frame with abnormal event detection
            break

        cv2.imshow("video", image)
        
        # Collect CPU and memory utilization
        cpu_percent_list.append(psutil.cpu_percent())
        memory_percent_list.append(psutil.virtual_memory().percent)

        print("Current Memory Usage:", (psutil.virtual_memory().percent)/2, "%")

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    # Calculate execution time
    end_time = time.time()
    execution_time = end_time - start_time

    # Print average CPU and memory utilization
    if cpu_percent_list:
        average_cpu_percent = sum(cpu_percent_list) / len(cpu_percent_list)
    else:
        average_cpu_percent = 0
    if memory_percent_list:
        average_memory_percent = sum(memory_percent_list) / len(memory_percent_list)
    else:
        average_memory_percent = 0
    print("Average CPU Utilization:", average_cpu_percent, "%")
    print("Average Memory Utilization:", average_memory_percent/2, "%")
    print("Execution Time:", execution_time, "seconds")

    cap.release()
    cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import time
import imutils
from tensorflow.keras.models import load_model
import psutil
import threading


def main():
    video_path = "D:/Omkar/Fourth Year/Major Project 2/Main/Avenue_Dataset/Avenue Dataset/testing_videos/pistol.avi"  # Update with your video file path

    # Create threads for each function
    weapon_thread = threading.Thread(target=weapon_detect, args=(video_path,))
    abnormal_thread = threading.Thread(target=abnormal_action, args=(video_path,))

    # Start both threads
    weapon_thread.start()
    abnormal_thread.start()

    # Wait for the weapon_detect thread to complete
    weapon_thread.join()

    # The weapon_detect thread has completed, so the main thread continues running

    # Print a message indicating the completion of weapon_detect
    print("Weapon detection thread has completed. Main thread continues...")

    # Wait for the abnormal_action thread to complete
    abnormal_thread.join()

    # The abnormal_action thread has completed, so the main thread continues running

    # Print a message indicating the completion of abnormal_action
    print("Abnormal action detection thread has completed. Main thread continues...")

    # Main thread exits

if __name__ == "__main__":
    main()


C:\Users\omkar\AppData\Roaming\Python\Python310\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 19 variables whereas the saved optimizer has 36 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.0005665372490945654
Abnormal Event Detected
Current Memory Usage: 31.3 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step
0.00056987376465916
Abnormal Event Detected
Current Memory Usage: 31.35 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 330ms/step
0.0006178227980892674
Abnormal Event Detected
Current Memory Usage: 31.3 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 327ms/step
0.0006155107115641277
Abnormal Event Detected
Current Memory Usage: 31.6 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
0.0006385845703340131
Abnormal Event Detected
Current Memory Usage: 31.6 %
weapon detected in frame
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step
0.0006088500525454147
Abnormal Event Detected
Current Memory Usage: 31.8 %
weapon detected in frame
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step
0.0005874589865164584
Abnormal Event Detected
Current Memory Usage: 32.0 %
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step
0.0006252787061573113
Abnormal Event Detected
Current Memory Usage: 32.0 %
weapon detected in frame
1/1 ━━━━━━━━━━━━━

In [3]:
def detection():
    val = weapon_detect(path)

    if val == 0:
        abnormal_action(path)
        print("weapon not detected but abnormal action detected")
    else:
        print("weapon detected")